Para replicar el referenciamiento de ImageNet en tu propio dataset de imágenes almacenado en Amazon S3 y ejecutar la transferencia de aprendizaje con VGG16, debes seguir varios pasos. Aquí tienes una guía general sobre cómo hacerlo:


In [1]:
# Librerías estándar
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

# TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Otras librerías
import os
import boto3
from PIL import Image
from sklearn.metrics import (accuracy_score, auc, average_precision_score, classification_report,
                             confusion_matrix, precision_score, recall_score)
from sklearn.model_selection import train_test_split

# Entrenamiento Animal y No Animal

In [2]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('ArchivosUtiles/trainingAnimal.pkl')
test = pd.read_pickle('ArchivosUtiles/testingAnimal.pkl')

In [3]:
X_train = train['Imagen']
y_train = train['Animal']
X_test = test['Imagen']
y_test = test['Animal']

In [4]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Descarga del modelo VGG16 pre-entrenado:
Descarga el modelo VGG16 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [5]:
input_shape = (150, 150, 3)

In [30]:
# Carga el modelo VGG16 preentrenado con pesos de ImageNet (no incluye las capas densas superiores)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

## Generador de datos:

Utiliza un generador de datos de Keras para cargar y preprocesar tus imágenes desde S3. Debes proporcionar la ruta a tus imágenes en S3 y etiquetas correspondientes. Aquí un ejemplo de cómo configurar un generador de datos:

In [31]:
# Número de clases en tu conjunto de datos
num_classes = len(y_train.unique())

In [32]:
# Tamaño del lote (batch size) que deseas utilizar durante el entrenamiento
batch_size = 32

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de VGG16 para que no se actualicen durante el entrenamiento.

In [39]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output) #Flatten output to 1 dimension

# # Agregar más capas densas
x = Dense(1024, activation='relu')(x)  # Añade una capa con Relu activation
x = Dropout(0.2)(x)  # Añade un dropout rate de 0.2

predictions = Dense(1, activation = 'sigmoid')(x)

In [40]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [41]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [49]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [51]:
# Entrena el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
151/151 [==============================] - 208s 1s/step - loss: 2.4253 - accuracy: 0.9534 - val_loss: 3.1730 - val_accuracy: 0.9338
Epoch 2/30
151/151 [==============================] - 199s 1s/step - loss: 1.5477 - accuracy: 0.9716 - val_loss: 3.4536 - val_accuracy: 0.9346
Epoch 3/30
151/151 [==============================] - 205s 1s/step - loss: 0.6468 - accuracy: 0.9727 - val_loss: 2.8843 - val_accuracy: 0.9421
Epoch 4/30
151/151 [==============================] - 204s 1s/step - loss: 0.8646 - accuracy: 0.9735 - val_loss: 2.8375 - val_accuracy: 0.9338
Epoch 5/30
151/151 [==============================] - 209s 1s/step - loss: 0.8134 - accuracy: 0.9756 - val_loss: 2.5009 - val_accuracy: 0.9371
Epoch 6/30
151/151 [==============================] - 206s 1s/step - loss: 0.7724 - accuracy: 0.9764 - val_loss: 2.2056 - val_accuracy: 0.9421
Epoch 7/30
151/151 [==============================] - 210s 1s/step - loss: 0.5277 - accuracy: 0.9812 - val_loss: 2.2880 - val_accuracy: 0.9379

In [54]:
# Guardar el modelo entrenado
model.save('ModelosFinales/modeloAnimalVGG16_v2.h5')

In [52]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

48/48 [==============================] - 57s 1s/step


In [53]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

# Accuracy: 0.9696714406065712
# Recall: 0.9854838709677419
# Specificity: 0.9523809523809523

# Accuracy: 0.9337748344370861
# Recall: 0.9396662387676509
# Specificity: 0.9274965800273598

Accuracy: 0.9456953642384106
Recall: 0.9614890885750963
Specificity: 0.9288645690834473


In [17]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

Average Precision (AP): 0.9728555826424565


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       731
           1       0.93      0.94      0.94       779

    accuracy                           0.93      1510
   macro avg       0.93      0.93      0.93      1510
weighted avg       0.93      0.93      0.93      1510



# Entrenamiento Guanaco y No Guanaco

In [83]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('ArchivosUtiles/trainingGuanaco.pkl')
test = pd.read_pickle('ArchivosUtiles/testingGuanaco.pkl')

In [84]:
X_train = train['Imagen']
y_train = train['Guanaco']
X_test = test['Imagen']
y_test = test['Guanaco']

In [85]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Balanceo

In [86]:
y_train.value_counts()

Guanaco
1.0    1786
0.0    1061
Name: count, dtype: int64

La base esta desbalanceada ya que existen muchos mas guanacos que otros animales. Para resolver esto se somete al resto de los animales a tecnicas de *Data Augmentation*

Data Augmentation: make training set larger by applying transformations. More information to learn from.
- Brightness and Contrast adjustments
- Rotations
- Gaussian noise
- Mirroring

In [81]:
def data_augmentation(image_tensor):

    # Convierte el tensor de imagen a una imagen TensorFlow
    image = tf.convert_to_tensor(image_tensor, dtype=tf.float32)

    # Brightness and Contrast adjustments
    if np.random.rand() < 0.8:
        image = tf.image.adjust_brightness(image, delta=0.2)  # Cambiar el brillo
        image = tf.image.adjust_contrast(image, contrast_factor=1.2)  # Cambiar el contraste

    # Rotations
    if np.random.rand() < 0.7:
        degrees = np.random.uniform(-10, 10)  # Rotación aleatoria entre -10 y 10 grados
        degrees = int(round(degrees))  # Redondea los grados a un entero
        image = tf.image.rot90(image, k=degrees // 90)  # Rotar la imagen

    # Gaussian noise
    if np.random.rand() < 0.2:
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1)
        image = image + noise

    # Mirroring (flip horizontal)
    if np.random.rand() < 0.5:
        image = tf.image.flip_left_right(image)

    # Convierte la imagen aumentada de nuevo a un tensor
    augmented_image_tensor = tf.convert_to_tensor(image.numpy(), dtype=tf.float32)

    return augmented_image_tensor

In [82]:
# Crea una nueva lista para almacenar los tensores de imágenes aumentados
imagen_tensor_aumentada = []

# Itera a través de las filas del DataFrame y aplica la función de aumento de datos
for index, row in train.iterrows():
    if np.random.rand() < 0.38:
        imagen_tensor = row['Imagen']
        imagen_aumentada = data_augmentation(imagen_tensor)
        imagen_tensor_aumentada.append(imagen_aumentada)

In [62]:
train_augmentation = pd.DataFrame(columns=['Imagen', 'Guanaco'])
train_augmentation['Imagen'] = imagen_tensor_aumentada
train_augmentation['Guanaco'] = False

In [63]:
train_augmentation= pd.concat([train, train_augmentation], ignore_index=True)

In [64]:
train_augmentation['Guanaco'].value_counts()

Guanaco
0.0    2141
1.0    1786
Name: count, dtype: int64

In [56]:
X_train = train_augmentation['Imagen']
y_train = train_augmentation['Guanaco']

In [57]:
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))

## Descarga del modelo VGG16 pre-entrenado:
Descarga el modelo VGG16 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [87]:
input_shape = (150, 150, 3)

In [88]:
# Carga el modelo VGG16 preentrenado con pesos de ImageNet (no incluye las capas densas superiores)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de VGG16 para que no se actualicen durante el entrenamiento.

In [111]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output)  # Flatten output to 1 dimension

# # Agregar más capas densas
# x = Dense(1024, activation='relu')(x)  # Añade una capa con Relu activation
# x = Dropout(0.2)(x)  # Añade un dropout rate de 0.2

# # Agregar más capas densas
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)

# # Agregar más capas densas
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)

# # Agregar más capas densas
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.2)(x)

# Capa de salida
predictions = Dense(1, activation='sigmoid')(x)

In [112]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [113]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [114]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [118]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [119]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [120]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
72/72 [==============================] - 146s 2s/step - loss: 2.1579 - accuracy: 0.6671 - val_loss: 1.5966 - val_accuracy: 0.6842
Epoch 2/30
72/72 [==============================] - 157s 2s/step - loss: 1.2139 - accuracy: 0.7519 - val_loss: 1.2077 - val_accuracy: 0.7193
Epoch 3/30
72/72 [==============================] - 168s 2s/step - loss: 0.8786 - accuracy: 0.7949 - val_loss: 1.0568 - val_accuracy: 0.7561
Epoch 4/30
72/72 [==============================] - 177s 2s/step - loss: 0.6680 - accuracy: 0.8274 - val_loss: 0.9972 - val_accuracy: 0.7632
Epoch 5/30
72/72 [==============================] - 138s 2s/step - loss: 0.5341 - accuracy: 0.8480 - val_loss: 0.8893 - val_accuracy: 0.7860
Epoch 6/30
72/72 [==============================] - 147s 2s/step - loss: 0.4234 - accuracy: 0.8704 - val_loss: 0.8472 - val_accuracy: 0.8000
Epoch 7/30
72/72 [==============================] - 162s 2s/step - loss: 0.3553 - accuracy: 0.8880 - val_loss: 0.8098 - val_accuracy: 0.8123
Epoch 8/30
72

In [101]:
# Guardar el modelo entrenado
model.save('ModelosFinales/modeloGuanacoVGG16_v2.h5')

In [121]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

23/23 [==============================] - 32s 1s/step


In [122]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Precision
precision = precision_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Precision: {precision}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

# Accuracy: 0.8932584269662921
# Recall: 0.9105145413870246
# Precision: 0.9187358916478555
# Specificity: 0.8641509433962264

Accuracy: 0.8735955056179775
Recall: 0.8970917225950783
Precision: 0.9011235955056179
Specificity: 0.8339622641509434


In [55]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

# Average Precision (AP): 0.9239772259039202

Average Precision (AP): 0.8610577537065705


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.39      0.96      0.56       138
        True       0.95      0.31      0.46       299

    accuracy                           0.51       437
   macro avg       0.67      0.64      0.51       437
weighted avg       0.77      0.51      0.49       437



In [ ]:
(y_pred == 0).sum() / len(y_pred)

0.7780320366132724

# Entrenamiento Categoria Especie

In [ ]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('ArchivosUtiles/trainingCategoria.pkl')
test = pd.read_pickle('ArchivosUtiles/testingCategoria.pkl')

In [ ]:
X_train = train['Imagen']
y_train = train['Categoria']
X_test = test['Imagen']
y_test = test['Categoria']

In [ ]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

# Entrenamiento Aislado

In [70]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('ArchivosUtiles/trainingAislado.pkl')
test = pd.read_pickle('ArchivosUtiles/testingAislado.pkl')

In [71]:
X_train = train['Imagen']
y_train = train['Aislado']
X_test = test['Imagen']
y_test = test['Aislado']

In [72]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Balanceo

In [73]:
y_train.value_counts()

Aislado
0    829
1    829
Name: count, dtype: int64

## Descarga del modelo VGG16 pre-entrenado:
Descarga el modelo VGG16 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [87]:
input_shape = (150, 150, 3)

In [88]:
# Carga el modelo VGG16 preentrenado con pesos de ImageNet (no incluye las capas densas superiores)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de VGG16 para que no se actualicen durante el entrenamiento.

In [89]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output)  # Flatten output to 1 dimension

# # Agregar más capas densas
# x = Dense(1024, activation='relu')(x)  # Añade una capa con Relu activation
# x = Dropout(0.2)(x)  # Añade un dropout rate de 0.2

# # Agregar más capas densas
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)

# # Agregar más capas densas
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)

# # Agregar más capas densas
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.2)(x)

# Capa de salida
predictions = Dense(1, activation='sigmoid')(x)

In [90]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [91]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [92]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [93]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [94]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
42/42 [==============================] - 99s 2s/step - loss: 2.8949 - accuracy: 0.6222 - val_loss: 2.4954 - val_accuracy: 0.6687
Epoch 2/30
42/42 [==============================] - 96s 2s/step - loss: 1.1120 - accuracy: 0.7624 - val_loss: 2.3405 - val_accuracy: 0.6837
Epoch 3/30
42/42 [==============================] - 116s 3s/step - loss: 0.6411 - accuracy: 0.8273 - val_loss: 2.5806 - val_accuracy: 0.6898
Epoch 4/30
42/42 [==============================] - 100s 2s/step - loss: 0.3683 - accuracy: 0.8869 - val_loss: 2.2491 - val_accuracy: 0.6747
Epoch 5/30
42/42 [==============================] - 217s 5s/step - loss: 0.2862 - accuracy: 0.8944 - val_loss: 2.4043 - val_accuracy: 0.6717
Epoch 6/30
42/42 [==============================] - 264s 6s/step - loss: 0.2074 - accuracy: 0.9216 - val_loss: 2.1060 - val_accuracy: 0.6777
Epoch 7/30
42/42 [==============================] - 219s 5s/step - loss: 0.2127 - accuracy: 0.9238 - val_loss: 2.3147 - val_accuracy: 0.7018
Epoch 8/30
42/4

In [97]:
# Guardar el modelo entrenado
model.save('ModelosFinales/modeloAisladoVGG16.h5')

In [95]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

13/13 [==============================] - 24s 2s/step


In [96]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Precision
precision = precision_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Precision: {precision}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

# Accuracy: 0.5927710843373494
# Recall: 0.5942028985507246
# Precision: 0.5913461538461539
# Specificity: 0.5913461538461539

Accuracy: 0.6265060240963856
Recall: 0.6811594202898551
Precision: 0.6130434782608696
Specificity: 0.5721153846153846


In [85]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

# Average Precision (AP): 0.9239772259039202

Average Precision (AP): 0.591230552516077


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.39      0.96      0.56       138
        True       0.95      0.31      0.46       299

    accuracy                           0.51       437
   macro avg       0.67      0.64      0.51       437
weighted avg       0.77      0.51      0.49       437

